In [7]:
# %pip install pandas
# %pip install numpy
# %pip install contractions
# %pip install bs4
# %pip install nltk
# %pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Importing Libraries

In [1]:
import pandas as pd
import gzip
import json
import re
from bs4 import BeautifulSoup
from unidecode import unidecode
import nltk
import contractions

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# from acronym import Acronym


# Df loading function

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  folder = parse(path)
  for d in folder:
    print(i)
    df[i] = d
    i += 1
  return df



In [ ]:
df = getDF('Electronics_5.json.gz')
data_frame = pd.DataFrame.from_dict(df, orient='index')
data_frame.to_csv('data_frame.csv', index=False)

In [ ]:
df = getDF('meta_Electronics.json.gz')
meta_df = pd.DataFrame.from_dict(df, orient='index')
meta_df.to_csv('meta_data.csv', index=False, escapechar='\\')

In [3]:
data = pd.read_csv("data_frame.csv")
meta_data = pd.read_csv("meta_data.csv")

/var/folders/4f/kjgrgc293tn7cn556xk1h1g40000gn/T/ipykernel_4806/1755724970.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data_frame.csv")


In [6]:
meta_data.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"['Electronics', 'Camera &amp; Photo', 'Video S...",NaN,['The following camera brands and models have ...,NaN,Genuine Geovision 1 Channel 3rd Party NVR IP S...,[],NaN,GeoVision,['Genuine Geovision 1 Channel NVR IP Software'...,"['>#3,092 in Tools &amp; Home Improvement &gt;...",[],Camera &amp; Photo,NaN,"January 28, 2014",$65.00,0011300000,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN
1,"['Electronics', 'Camera &amp; Photo']",NaN,"[""This second edition of the Handbook of Astro...",NaN,"Books ""Handbook of Astronomical Image Processi...",['0999470906'],NaN,33 Books Co.,['Detailed chapters cover these fundamental to...,"['>#55,933 in Camera &amp; Photo (See Top 100 ...","['0943396670', '1138055360', '0999470906']",Camera &amp; Photo,NaN,"June 17, 2003",NaN,0043396828,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN
2,"['Electronics', 'eBook Readers &amp; Accessori...",NaN,['A zesty tale. (Publishers Weekly)<br /><br /...,NaN,One Hot Summer,"['0425167798', '039914157X']",NaN,Visit Amazon's Carolina Garcia Aguilera Page,[],"3,105,177 in Books (",[],Books,NaN,NaN,$11.49,0060009810,[],[],NaN
3,"['Electronics', 'eBook Readers & Accessories',...",NaN,[],NaN,Hurray for Hattie Rabbit: Story and pictures (...,"['0060219521', '0060219580', '0060219394']",NaN,Visit Amazon's Dick Gackenbach Page,[],"2,024,298 in Books (","['0060219521', '0060219475', '0060219394']",Books,NaN,NaN,.a-section.a-spacing-mini{margin-bottom:6px!im...,0060219602,[],[],NaN
4,"['Electronics', 'eBook Readers & Accessories',...",NaN,['&#8220;sex.lies.murder.fame. is brillllli&#8...,NaN,sex.lies.murder.fame.: A Novel,[],NaN,Visit Amazon's Lolita Files Page,[],"3,778,828 in Books (",[],Books,NaN,NaN,$13.95,0060786817,[],[],NaN


In [7]:
product_indices = []
product = 'Headphones'
for i in range(len(meta_data)):
    if product in meta_data['category'][i]:
        product_indices.append(i)

len(product_indices)

31305

In [8]:
product_meta_data_df = meta_data.iloc[product_indices]
product_meta_data_df = product_meta_data_df.dropna(subset=['title', 'asin'])
product_meta_data_df = product_meta_data_df.drop_duplicates()


In [10]:
product_df= pd.merge(data, product_meta_data_df, on='asin', how='inner')
product_df = product_df.dropna(subset=['overall'])
product_df = product_df.drop_duplicates()


In [ ]:
product_df.info()



In [ ]:
total_reviews = product_df['reviewText'].count()
average_rating = product_df['overall'].mean()
num_unique_products = product_df['asin'].nunique()
good_ratings = product_df[product_df['overall'] >= 3]['overall'].count()
bad_ratings = product_df[product_df['overall'] < 3]['overall'].count()
rating_counts = product_df['overall'].value_counts().sort_index()

# Display the results
print(" . Number of rows for the product:", len(product_df))
print("a. Number of Reviews:", total_reviews)
print("b. Average Rating Score:", average_rating)
print("c. Number of Unique Products:", num_unique_products)
print("d. Number of Good Ratings:", good_ratings)
print("e. Number of Bad Ratings:", bad_ratings)
print("f. Number of Reviews corresponding to each Rating:")
print(rating_counts)


In [ ]:

# # Function to expand acronyms (you can extend this as needed)
def expand_acronyms(text):
    expanded_words = []    
    for word in text.split():
        expanded_words.append(contractions.fix(word))  
    expanded_text = ' '.join(expanded_words)
    return expanded_text
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
def remove_accented_chars(text):
    return unidecode(text)
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)
def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text
def normalize_text(text):
    text = text.lower()
    text = expand_acronyms(text)
    text = remove_html_tags(text)
    text = remove_accented_chars(text)
    text = remove_special_characters(text)
    text = lemmatize_text(text)
    return text

product_df['reviewText'] = product_df['reviewText'].astype(str)
for i in range(len(product_df)):
    pre_text =  normalize_text(product_df['reviewText'].iloc[i])
    product_df['reviewText'].iloc[i] =pre_text


In [28]:
top_20_most_reviewed_brands = product_df['brand'].value_counts().head(20)
top_20_least_reviewed_brands = product_df['brand'].value_counts().tail(20)
most_positively_reviewed_headphone = product_df.groupby('title')['overall'].mean().nlargest(1)


print("a. Top 20 most reviewed brands:")
print(top_20_most_reviewed_brands)
print("b. Top 20 least reviewed brands:")
print(top_20_least_reviewed_brands)
print("c. Most positively reviewed headphone:")
print(most_positively_reviewed_headphone)

Most positively reviewed headphone:
title
Myron &amp; Davis AE52 Dual Channel IR headphone    5.0
Name: overall, dtype: float64
a. Top 20 most reviewed brands:
brand
Sony              31260
Sennheiser        21221
Bose               9489
Plantronics        8239
Skullcandy         7862
JVC                7024
JLAB               6977
Audio-Technica     6651
Philips            6458
Panasonic          5980
Koss               5764
Mpow               5643
LG                 5624
Samsung            5597
Bluedio            5364
MEE audio          4644
Anker              4291
Symphonized        4285
TaoTronics         4056
Beats              3993
Name: count, dtype: int64
Top 20 least reviewed brands:
brand
EMPIRE AUDIO USA         5
Cosa Nova                5
MAXELL(R)                5
OAproda                  5
TomTom                   5
tech21                   5
SoulBuddy                5
fizwan                   5
OCR                      5
Spark                    5
VIILER                

In [ ]:
set(product_df['date'])

In [35]:
product_df['date'] = pd.to_datetime(product_df['date'], errors='coerce')
# Count of ratings for the product over 5 consecutive years
ratings_over_years = product_df.set_index('date').groupby(pd.Grouper(freq='Y')).size().head(5)
print("Count of ratings for the product over 5 consecutive years:")
print(ratings_over_years)

/var/folders/4f/kjgrgc293tn7cn556xk1h1g40000gn/T/ipykernel_3520/1072341768.py:3: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  ratings_over_years = product_df.set_index('date').groupby(pd.Grouper(freq='Y')).size().head(5)


Count of ratings for the product over 5 consecutive years:
date
1973-12-31    129
1974-12-31      0
1975-12-31      0
1976-12-31      0
1977-12-31      0
dtype: int64


In [60]:


# Create an instance of the Acronym class
acronyms = Acronym()

# Expand acronyms
text = "lol, omg, btw"
expanded_text = acronyms.expand(text)

print(expanded_text)


NameError: name 'Acronym' is not defined